# Setup

In [ ]:
# Run this locally. Binder dont need run.
!conda remove -n ICT3104 --all -y

In [ ]:
# Run this locally. Binder dont need run.
!conda env create -n ICT3104 --file environment.yml

In [ ]:
# Run this locally. Binder dont need run.
!conda activate ICT3104

In [ ]:
# Ensure environment is working
import torch
import timm
import sklearn
import numpy

# Example Use

# Apply Module